In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
train_path = r'C:\Users\HP\OneDrive\Desktop\deep_learning_lab\sign_language\dataset\train'
valid_path = r'C:\Users\HP\OneDrive\Desktop\deep_learning_lab\sign_language\dataset\valid'
test_path = r'C:\Users\HP\OneDrive\Desktop\deep_learning_lab\sign_language\dataset\test'

In [ ]:
train_batches = ImageDataGenerator().flow_from_directory(train_path,
                                                        target_size=(244, 244),
                                                        batch_size=10)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path,
                                                        target_size=(244, 244),
                                                        batch_size=30)
test_batches = ImageDataGenerator().flow_from_directory(test_path,
                                                        target_size=(244, 244),
                                                        batch_size=50,
                                                        shuffle=False)

In [ ]:
base_model = ResNet50(weights = "imagenet", include_top = False, input_shape= (244, 244,3), pooling = 'avg')
base_model.summary()

In [ ]:
for layer in base_model.layers[:-5]:
    layer.trainable = False

base_model.summary()

In [ ]:
x = Dense(10, activation='softmax', name='softmax')(base_model.output)
model = Model(inputs=base_model.input, outputs=x)
model.summary()

In [ ]:
model.compile(Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:

checkpoint = ModelCheckpoint('signlanguage.keras')

# Train the model
history = model.fit(
    train_batches, 
    validation_data=valid_batches,
    epochs=20, 
    callbacks=[checkpoint]
)

In [ ]:
model = load_model('signlanguage.keras')

In [ ]:
def predict_sign(img_path):
    # Load and preprocess image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    # Make prediction
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    class_labels = list(train_batches.class_indices.keys())
    predicted_label = class_labels[predicted_class]
    
    # Display result
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Predicted: {predicted_label}')
    plt.show()
    
    return predicted_label

In [ ]:
predicted_label = predict_sign('C:\\Users\\HP\\OneDrive\\Desktop\\deep_learning_lab\\sign_language\\dataset\\test\\A\\A1.jpg')
print(f'Predicted label: {predicted_label}')